In [1]:
import os

In [2]:
def get_file_contents(filename):
    """ Given a filename,
        return the contents of that file
    """
    try:
        with open(filename, 'r') as f:
            # It's assumed our file contains a single line,
            # with our API key
            return f.read().strip()
    except FileNotFoundError:
        print("'%s' file not found" % filename)

In [ ]:
filename = "..\\GoogleAPIKey.txt"
os.environ['GOOGLE_API_KEY'] = get_file_contents(filename)
filename_grokKey = "..\\GroqAPIKey.txt"
os.environ['GROQ_API_KEY'] = get_file_contents(filename_grokKey)

### Agents

In [35]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.tools import DuckDuckGoSearchResults
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_groq import ChatGroq

In [36]:
#llm= ChatGoogleGenerativeAI(model='gemini-1.5-pro-001')
llm = ChatGroq(model="llama-3.1-8b-instant")

In [29]:
search = DuckDuckGoSearchResults()
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

In [30]:
prompt = ChatPromptTemplate(
    [
        (
            "system","You are a helpful assistant. For answering the user query, look for information using DuckDuckGo Search and Wikipedia and then give the final result."
        ),
        ("placeholder","{chat_history}"),
        ("human","{input}"),
        ("placeholder","{agent_scratchpad}"), # must add when using tool calling agent
    ]
)

In [31]:
tools = [search,wikipedia]

In [38]:
agent = create_tool_calling_agent(llm,tools,prompt)

In [39]:
agent_executor = AgentExecutor(agent=agent,tools=tools, verbose=True)

In [40]:
agent_executor.invoke({"input":"what's the weather  in Singapore today"})



> Entering new AgentExecutor chain...

Invoking: `duckduckgo_results_json` with `{'query': 'Singapore weather today'}`


snippet: Get the hourly weather forecast for today in Singapore, including temperature, chance of rain, wind, humidity and more. See the sunrise and sunset times, UV index and clothing recommendation for Singapore., title: Singapore local weather (live): today, hourly weather, link: https://www.weather25.com/asia/singapore?page=today, snippet: Singapore Weather Forecasts. Weather Underground provides local & long-range weather forecasts, weatherreports, maps & tropical weather conditions for the Singapore area., title: Singapore, Singapore Weather Conditions - Weather Underground, link: https://www.wunderground.com/weather/sg/singapore, snippet: Today, in Singapore, primarily cloud-filled and occasionally unstable weather is anticipated. With minimal precipitation of 1mm, rain is predicted from 13h to 15h. The temperature will be between the lowest temperature of 2

{'input': "what's the weather  in Singapore today",
 'output': 'The weather in Singapore today is primarily cloud-filled and occasionally unstable, with minimal precipitation of 1mm predicted from 13h to 15h. The temperature will be between the lowest temperature of 28°C and the highest temperature of 31°C.'}

In [41]:
agent_executor.invoke({"input":"what's the weather  in Delhi today"})



> Entering new AgentExecutor chain...

Invoking: `duckduckgo_results_json` with `{'query': 'Delhi weather today'}`


snippet: Today's Weather in New Delhi: In New Delhi today, the weather is expected to be Fog with a maximum temperature of 16°C and a minimum of 11°C. Sunrise in New Delhi is set for 07:15 AM and ..., title: New Delhi Weather Forecast 3 May 2025 - Times of India, link: https://timesofindia.indiatimes.com/weather/new-delhi-weather-forecast-today/110011, snippet: New Delhi, India - Current temperature and weather conditions. Detailed hourly weather forecast for today - including weather conditions, temperature, pressure, humidity, precipitation, dewpoint, wind, visibility, and UV index data., title: Weather today - New Delhi, India, link: https://www.weather-atlas.com/en/india/new-delhi, snippet: Delhi Weather Today Live: Delhi Airport issues traffic advisory. It said, "Due to inclement weather conditions and thunderstorms in Delhi, some flights have been impacted at Del

{'input': "what's the weather  in Delhi today",
 'output': 'The current weather in Delhi is expected to be foggy with a maximum temperature of 16°C and a minimum of 11°C. The weather forecast for today is as follows:\n\n* Maximum temperature: 16°C\n* Minimum temperature: 11°C\n* Sunrise: 07:15 AM\n* Sunset: 06:30 PM\n* Fog: Yes\n* Rain: No\n\nIt is recommended to carry warm clothing and an umbrella to stay comfortable throughout the day.'}

In [43]:
agent_executor.invoke({"input":"When was Nelson Mandela born and what are some of his famous quotes?"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Nelson Mandela birth and famous quotes'}`


Page: Nelson Mandela
Summary: Nelson Rolihlahla Mandela ( man-DEL-ə, Xhosa: [xolíɬaɬa mandɛ̂ːla]; born Rolihlahla Mandela; 18 July 1918 – 5 December 2013) was a South African anti-apartheid activist and politician who served as the first president of South Africa from 1994 to 1999. He was the country's first black head of state and the first elected in a fully representative democratic election. His government focused on dismantling the legacy of apartheid by fostering racial reconciliation. Ideologically an African nationalist and socialist, he served as the president of the African National Congress (ANC) party from 1991 to 1997.
A Xhosa, Mandela was born into the Thembu royal family in Mvezo, South Africa. He studied law at the University of Fort Hare and the University of Witwatersrand before working as a lawyer in Johannesburg. There he became involved in ant

{'input': 'When was Nelson Mandela born and what are some of his famous quotes?',
 'output': 'Nelson Mandela was born on July 18, 1918. Some of his famous quotes include:\n\n* "Education is the most powerful weapon which you can use to change the world."\n* Nelson Mandela wasn\'t just a person; he was a person—a freedom fighter, president, and full-time inspiration.\n* Nelson Rolihlahla Mandela (born July 18, 1918, in South Africa - died December 5, 2013, in Johannesburg, South Africa) was the first President of South Africa elected in a fully representative democratic election, co-winner of the Nobel Peace Prize with F.W. de Klerk, revolutionary, anti-apartheid icon, and philanthropist who his whole life dedicated to the struggle for human rights.\n* Nelson Mandela exemplifies the strength of character, resilience, and commitment to justice that can inspire the world.\n* "The greatest glory in living lies not in never falling, but in rising every time we fall."'}